<h1 align="center">Topic: Building a Text Summarizer</h1>

<h2>Importing required libraries</h2>

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en import English
import numpy as np

<h2>Load spacy model for sentence tokenization</h2>

In [2]:
nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer'))

In [3]:
text_corpus = """
Google celebrated British illustrator and artist Sir John Tenniel's 
200th birth anniversary with a doodle on February 28. An acclaimed 
Victorian painter, Tenniel is celebrated for his illustrations for 
Lewis Carroll's Alice's Adventures in Wonderland and Through the Looking-Glass.
Tenniel was born in Bayswater, West London in 1820. At the age of 20, Tenniel 
received a major eye injury and eventually, lost sight in his right eye. 
From a very early age, Tenniel was appreciated as a humorist and soon after, 
also cultured his talent for scholarly caricature.
His first illustration was for Samuel Carter Hall's The Book of British 
Ballads in 1842. Eight years later, he joined the historic weekly magazine 
Punch as a political cartoonist. Lewis Carroll noticed Tenniel's distinct style 
of work and in 1864, approached the artist to illustrate his book, Alice's 
Adventures in Wonderland. This association marked Carroll and Tenniel's creative 
partnership and continued with Through the Looking Glass in 1872. "The result: 
a series of classic characters, such as Alice and the Cheshire Cat, as depicted 
in the Doodle artwork's rendition of their iconic meeting-characters who, along 
with many others, remain beloved by readers of all ages to this day," the Google 
Doodle page says. After working with Lewis Carroll, Tenniel resumed his work with 
Punch. For his work, Tenniel also received a knighthood in 1893.
Sir John Tenniel died on February 25, 1914. He was 93.
"""

<h2>Create spacy document for further sentence level tokenization</h2>

In [4]:
doc = nlp(text_corpus.replace("\n", ""))
sentences = [sent.string.strip() for sent in doc.sents]

<h2>Peeking into our tokenized sentences</h2>

In [5]:
print("Senetence are: \n", sentences)

Senetence are: 
 ["Google celebrated British illustrator and artist Sir John Tenniel's 200th birth anniversary with a doodle on February 28.", "An acclaimed Victorian painter, Tenniel is celebrated for his illustrations for Lewis Carroll's Alice's Adventures in Wonderland and Through the Looking-Glass.", 'Tenniel was born in Bayswater, West London in 1820.', 'At the age of 20, Tenniel received a major eye injury and eventually, lost sight in his right eye.', 'From a very early age, Tenniel was appreciated as a humorist and soon after, also cultured his talent for scholarly caricature.', "His first illustration was for Samuel Carter Hall's The Book of British Ballads in 1842.", 'Eight years later, he joined the historic weekly magazine Punch as a political cartoonist.', "Lewis Carroll noticed Tenniel's distinct style of work and in 1864, approached the artist to illustrate his book, Alice's Adventures in Wonderland.", 'This association marked Carroll and Tenniel\'s creative partnership 

<h2>Creating sentence organizer</h2>

In [6]:
# Let's create an organizer which will store the sentence ordering to later reorganize the 
# scored sentences in their correct order
sentence_organizer = {k:v for v,k in enumerate(sentences)}

<h2>Peeking into our sentence organizer</h2>

In [7]:
print("Our sentence organizer: \n", sentence_organizer)

Our sentence organizer: 
 {"Google celebrated British illustrator and artist Sir John Tenniel's 200th birth anniversary with a doodle on February 28.": 0, "An acclaimed Victorian painter, Tenniel is celebrated for his illustrations for Lewis Carroll's Alice's Adventures in Wonderland and Through the Looking-Glass.": 1, 'Tenniel was born in Bayswater, West London in 1820.': 2, 'At the age of 20, Tenniel received a major eye injury and eventually, lost sight in his right eye.': 3, 'From a very early age, Tenniel was appreciated as a humorist and soon after, also cultured his talent for scholarly caricature.': 4, "His first illustration was for Samuel Carter Hall's The Book of British Ballads in 1842.": 5, 'Eight years later, he joined the historic weekly magazine Punch as a political cartoonist.': 6, "Lewis Carroll noticed Tenniel's distinct style of work and in 1864, approached the artist to illustrate his book, Alice's Adventures in Wonderland.": 7, 'This association marked Carroll and

<h2>Creating TF-IDF model</h2>

In [8]:
# Let's now create a tf-idf (Term frequnecy Inverse Document Frequency) model
tf_idf_vectorizer = TfidfVectorizer(min_df=2,  max_features=None, 
                                    strip_accents='unicode', 
                                    analyzer='word',
                                    token_pattern=r'\w{1,}',
                                    ngram_range=(1, 3), 
                                    use_idf=1,smooth_idf=1,
                                    sublinear_tf=1,
                                    stop_words = 'english')

In [9]:
# Passing our sentences treating each as one document to TF-IDF vectorizer
tf_idf_vectorizer.fit(sentences)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=2, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=1, stop_words='english', strip_accents='unicode',
                sublinear_tf=1, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=1, vocabulary=None)

In [10]:
# Transforming our sentences to TF-IDF vectors
sentence_vectors = tf_idf_vectorizer.transform(sentences)

<h2>Performing sentence scoring</h2>

In [11]:
# Getting sentence scores for each sentences
sentence_scores = np.array(sentence_vectors.sum(axis=1)).ravel()

# Sanity checkup
print(len(sentences) == len(sentence_scores))

True


In [12]:
# Getting top-n sentences
N = 3
top_n_sentences = [sentences[ind] for ind in np.argsort(sentence_scores, axis=0)[::-1][:N]]


<h2>Performing final summarization</h2>

In [13]:
# Let's now do the sentence ordering using our prebaked sentence_organizer
# Let's map the scored sentences with their indexes
mapped_top_n_sentences = [(sentence,sentence_organizer[sentence]) for sentence in top_n_sentences]
print("Our top_n_sentence with their index: \n")
for element in mapped_top_n_sentences:
    print(element)

# Ordering our top-n sentences in their original ordering
mapped_top_n_sentences = sorted(mapped_top_n_sentences, key = lambda x: x[1])
ordered_scored_sentences = [element[0] for element in mapped_top_n_sentences]

# Our final summary
summary = " ".join(ordered_scored_sentences)

Our top_n_sentence with their index: 

("An acclaimed Victorian painter, Tenniel is celebrated for his illustrations for Lewis Carroll's Alice's Adventures in Wonderland and Through the Looking-Glass.", 1)
("Lewis Carroll noticed Tenniel's distinct style of work and in 1864, approached the artist to illustrate his book, Alice's Adventures in Wonderland.", 7)
("Google celebrated British illustrator and artist Sir John Tenniel's 200th birth anniversary with a doodle on February 28.", 0)


<h2>Result / Summary</h2>

In [14]:
print("Summary: \n", summary)

Summary: 
 Google celebrated British illustrator and artist Sir John Tenniel's 200th birth anniversary with a doodle on February 28. An acclaimed Victorian painter, Tenniel is celebrated for his illustrations for Lewis Carroll's Alice's Adventures in Wonderland and Through the Looking-Glass. Lewis Carroll noticed Tenniel's distinct style of work and in 1864, approached the artist to illustrate his book, Alice's Adventures in Wonderland.


<h1>Creating a function template using above steps:</h1>

In [15]:
def summarizer(text, tokenizer, max_sent_in_summary=3):
    # Create spacy document for further sentence level tokenization
    doc = nlp(text_corpus.replace("\n", ""))
    sentences = [sent.string.strip() for sent in doc.sents]
    # Let's create an organizer which will store the sentence ordering to later reorganize the 
    # scored sentences in their correct order
    sentence_organizer = {k:v for v,k in enumerate(sentences)}
    # Let's now create a tf-idf (Term frequnecy Inverse Document Frequency) model
    tf_idf_vectorizer = TfidfVectorizer(min_df=2,  max_features=None, 
                                        strip_accents='unicode', 
                                        analyzer='word',
                                        token_pattern=r'\w{1,}',
                                        ngram_range=(1, 3), 
                                        use_idf=1,smooth_idf=1,
                                        sublinear_tf=1,
                                        stop_words = 'english')
    # Passing our sentences treating each as one document to TF-IDF vectorizer
    tf_idf_vectorizer.fit(sentences)
    # Transforming our sentences to TF-IDF vectors
    sentence_vectors = tf_idf_vectorizer.transform(sentences)
    # Getting sentence scores for each sentences
    sentence_scores = np.array(sentence_vectors.sum(axis=1)).ravel()
    # Getting top-n sentences
    N = max_sent_in_summary
    top_n_sentences = [sentences[ind] for ind in np.argsort(sentence_scores, axis=0)[::-1][:N]]
    # Let's now do the sentence ordering using our prebaked sentence_organizer
    # Let's map the scored sentences with their indexes
    mapped_top_n_sentences = [(sentence,sentence_organizer[sentence]) for sentence in top_n_sentences]
    # Ordering our top-n sentences in their original ordering
    mapped_top_n_sentences = sorted(mapped_top_n_sentences, key = lambda x: x[1])
    ordered_scored_sentences = [element[0] for element in mapped_top_n_sentences]
    # Our final summary
    summary = " ".join(ordered_scored_sentences)
    return summary

In [16]:
print("Summarizer Result: \n", summarizer(text=text_corpus, tokenizer=nlp, max_sent_in_summary=3))

Summarizer Result: 
 Google celebrated British illustrator and artist Sir John Tenniel's 200th birth anniversary with a doodle on February 28. An acclaimed Victorian painter, Tenniel is celebrated for his illustrations for Lewis Carroll's Alice's Adventures in Wonderland and Through the Looking-Glass. Lewis Carroll noticed Tenniel's distinct style of work and in 1864, approached the artist to illustrate his book, Alice's Adventures in Wonderland.


<h1 align="center">Understanding TF-IDF</h1>

In [4]:
from collections import Counter
import pandas as pd


documentA = 'the man went out for a walk'
documentB = 'the children sat around the fire'

baga = documentA.split(" ")
bagb = documentB.split(" ")

unq = list(set(baga).union(set(bagb)))

a = Counter(baga)
b = Counter(bagb)

row1 = [a.get(i, 0)/len(baga) for i in unq]
row2 = [b.get(i, 0)/len(bagb) for i in unq]

d = pd.DataFrame([row1, row2], columns=unq)

d

,for,sat,around,fire,a,the,man,went,out,walk,children
0,0.142857,0.000000,0.000000,0.000000,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.000000
1,0.000000,0.166667,0.166667,0.166667,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.166667


In [5]:
import numpy as np


documentA = 'the man went out for a walk'
documentB = 'the children sat around the fire'

baga = documentA.split(" ")
bagb = documentB.split(" ")

unq = list(set(baga).union(set(bagb)))

a = Counter(baga)
b = Counter(bagb)

row1 = [a.get(i, 0)/len(baga) for i in unq]
row2 = [b.get(i, 0)/len(bagb) for i in unq]

docf = {t:((1 if t in a else 0)+(1 if t in b else 0)) for t in unq}

row1 = [v*np.log(2/docf.get(i)) for i,v in zip(unq, row1)]
row2 = [v*np.log(2/docf.get(i)) for i,v in zip(unq, row2)]

d = pd.DataFrame([row1, row2], columns=unq)

d

,for,sat,around,fire,a,the,man,went,out,walk,children
0,0.099021,0.000000,0.000000,0.000000,0.099021,0.0,0.099021,0.099021,0.099021,0.099021,0.000000
1,0.000000,0.115525,0.115525,0.115525,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.115525


In [6]:
d.loc[:, "score"] = d.values.sum(axis=1)
d

,for,sat,around,fire,a,the,man,went,out,walk,children,score
0,0.099021,0.000000,0.000000,0.000000,0.099021,0.0,0.099021,0.099021,0.099021,0.099021,0.000000,0.594126
1,0.000000,0.115525,0.115525,0.115525,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.115525,0.462098
